# Computer Vision with Amazon Rekognition Custom Labels 

***
Welcome to an end-to-end example of how to use [Amazon Rekognition Custom Labels](https://aws.amazon.com/rekognition/custom-labels-features/) to create different types of computer vision models. 

Custom Labels is a fully managed Amazon Rekognition service that lets you build custom computer vision models that are unique to your business, even if you have little or no machine learning (ML) expertise. With Custom Labels you can create computer vision models for tasks such as classifying images or finding the location of unique objects on an image. Custom Labels builds on Rekognition’s existing capabilities, which are already trained on tens of millions of images across many categories. It abstracts much of the complexity required to build a computer vision model. Instead of thousands of images, you can use a small set of training images (typically a few hundred images or less) that are specific to your use case. Behind the scenes, Custom Labels automatically loads and inspects the training data, selects the right machine learning algorithms, trains your model, finds the optimal hyper-parameters, tests the model, and provides model performance metrics. Amazon Rekognition Custom Labels also provides an easy to use AWS console for the entire ML workflow, including labeling images, training, deploying a model, and visualizing the test results. 

In this notebook, we demonstrate how to use [Custom Label API](https://docs.aws.amazon.com/rekognition/latest/customlabels-dg/custom-labels-api-reference.html) to create an image classification model, a multi-label classification model, and an object detection model. We use the Custom Labels [Getting started](https://docs.aws.amazon.com/rekognition/latest/customlabels-dg/getting-started.htmltraining) datasets to provide training and test datasets for each model type:
* ***Image classification model - Rooms dataset***. Images labeled with different types of rooms or locations in a house. There is one label per image, such as living_room. The trained model classifies images from the set of labels used to train the model. A typical use case is a real estate website. House searches can be easily updated by using an image classification model to automatically classify images of living rooms, backyards, bedrooms, and other household locations.  
* ***Multi-label classification model - Flowers dataset***. Images labeled with different categories of flower type and flower attributes, such as the growth stage of the plant.  A single image is labeled with multiple categories, such as helleborus (flower type) and not_fully_grown (growth stage). The trained model categorizes images according to the categories used to train the model. An example use case is plant growers using a multi-label classification model to distinguish between different types of flowers and if the flowers are ready for market (fully grown).
* ***Object detection model - PCB dataset***. Images labeled with circuit board parts and their locations on the image. The trained model finds the locations and types of object detected in an image, based on the object types defined in the training dataset. An example use case is the electronics industry, where an object detection model can help count the number of capacitors on a circuit board.
* ***Brand/Logo detection model - AWS services dataset***. Images labeled with different types of AWS logos such as Amazon Textract, and AWS lambda. The trained model finds the locations of the logos or brands in an image, based on the label used to train the model. An example use case is in the media industry, where an object detection model can help to report on the location of sponsor logos in photographs.

Custom Labels determines the type of model that it trains based on the training dataset. Since the Custom Label API is identical for each model type, you can use the notebook for image classification, multi-label classification, or object detection, without changing the code. You only need to change the dataset that you use to train the model.

This notebook requires no ML expertise to train a model with the example dataset or with your own business specific dataset. You can use the API operations discussed in this notebook in your own applications. This notebook is based on [What Is Amazon Rekognition Custom Labels?](https://docs.aws.amazon.com/rekognition/latest/customlabels-dg/what-is.html).


***

## Outline

1. [Prerequisites and Set up](#1.-Prerequisites-and-setup)
    1. [Set up IAM permissions](#1.1.-Set-up-IAM-permissions)
    2. [Create a bucket and set up permissions](#1.2.-Create-a-bucket-and-set-up-permissions)
    3. [Install dependencies](#1.3.-Install-dependencies)
2. [Dataset Set Up](#2.-Dataset-Set-Up)
    1. [Select and download the Dataset](#2.1.-Select-and-download-the-Dataset)
    2. [Substitute Manifest file and upload the dataset](#2.2.-Substitute-Manifest-file-and-upload-the-dataset)
    3. [Importing train and test dataset](#2.3.-Importing-train-and-test-dataset)
3. [Train the model](#3.-Train-the-model)
    1. [Start Training](#3.1.-Start-Training)
    2. [Retrieve the trained model metrics](#3.2.-Retrieve-the-trained-model-metrics)
4. [Start model and run inference](#4.-Start-model-and-run-inference)
    1. [Start the trained model](#4.1.-Start-the-trained-model)
    2. [Run tests on the validation images](#4.2.-Run-tests-on-the-validation-images)
    3. [Stop the trained model](#4.3.-Stop-the-trained-model)
5. [Conclusion](#5.-Conclusion)
6. [Learn more about Rekognition Custom Labels](#6.-Learn-more-about-Rekognition-Custom-Labels)

## 1. Prerequisites and setup

***
Amazon Rekognition Custom Labels requires certain access policies that are attached to the IAM role and the Amazon S3 bucket that stores the datasets. By default, these policies are not present in your Amazon SageMaker Studio environment. We show how to set these permissions,  install dependencies, and set up relevant environment variables.
***

### 1.1. Set up IAM permissions

***

To use the Amazon Rekognition Custom Label APIs in Sagemaker Studio, we need to attach <b>AmazonRekognitionCustomLabelsFullAccess</b> and <b>AmazonS3FullAccess</b> policies to the IAM role associated with the Sagemaker Studio user. These policies provides full access to Amazon Rekognition Custom Labels and to Amazon S3. 

The Custom Labels policies cannot be attached to the current IAM role from inside the notebook. You can attach the policies by using the IAM console or by using AWS CloudShell. We show how to attach the policies by using CloudShell. For information on how to attach the policies with the IAM console, see [example notebook](https://github.com/aws-samples/amazon-rekognition-code-samples/blob/main/custom-labels/custom-object-detection-using-rekognition-custom-labels-api.ipynb).

**Note:** For a production application, we recommend that you restrict access policies to only those needed to run the application. Permissions can be restricted based on the use case (training/inference) and specific resource names (such as a full S3 bucket name or an S3 bucket name pattern). You should also restrict access to the Custom Labels or Amazon Sagemaker operations to just those that your application needs. For more information, See [Set up Amazon Rekognition Custom Labels permissions](https://docs.aws.amazon.com/rekognition/latest/customlabels-dg/su-console-policy.html).


**To get the IAM command**
1. Run the following cell to get the command that attaches the policies to the IAM role.
2. Note the output from the command. You need it in the next step.
***

In [ ]:
# First, we start by getting the IAM role associated with the studio instance running the current notebook. 
import sagemaker

import boto3

sts_client = boto3.client("sts")
arn = sts_client.get_caller_identity().get("Arn")

role_name = arn.split("/")[1]

# Command to attach AmazonRekognitionFullAccess to the IAM role.
cmd_1 = f"aws iam attach-role-policy --policy-arn arn:aws:iam::aws:policy/AmazonRekognitionFullAccess --role-name {role_name} "

# Command to attach AmazonS3FullAccess to the IAM role.
cmd_2 = f"aws iam attach-role-policy --policy-arn arn:aws:iam::aws:policy/AmazonS3FullAccess --role-name {role_name}"

cmd = f"{cmd_1} && {cmd_2}"

# Next, we print the commands which attach the required role.
print("\x1b[0;39;43m" + "Please copy the following command and execute in CloudShell:" + "\x1b[0m")
print(cmd)

### **Attach IAM policies by using AWS CloudShell**

***
You can attach the required policies by running a command from any shell authenticated with the credentials of the current AWS account. For simplicity, we use the AWS CloudShell service. 

**To attach the IAM policies**

1. Open the [CloudShell console](https://aws.amazon.com/cloudshell/) in the same AWS account as the current Sagemaker Studio domain. It might take up to a minute to start. Note that the IAM role used in Cloudshell must be able to change IAM permissions. Typically, the Administrator role can change IAM permissions. 

![cloudshell_part_1.png](https://jumpstart-cache-prod-us-east-1.s3.us-east-1.amazonaws.com/ai_services_assets/custom_labels/screenshots/cloudshell_screenshot_part_1.png)

2. At the CloudShell command prompt, run the command that you noted in the output from the previous cell.

![cloudshell_part_2.jpeg](https://jumpstart-cache-prod-us-east-1.s3.us-east-1.amazonaws.com/ai_services_assets/custom_labels/screenshots/cloudshell_screenshot_part_2.jpeg)

***

### 1.2. Create a bucket and set up permissions
***
To read training and test images, Amazon Rekognition Custom Labels needs permissions to read the S3 bucket that contains the images. We show how to add the permissions to the bucket. We start by creating an S3 bucket (if it doesn't exist) and then set the bucket permissions that Amazon Rekognition Custom Labels needs. 

#### 1.2.1. Creating an S3 bucket

We create an S3 bucket to host the dataset. The Custom Labels API and the SageMaker Image Classification algorithm API use the bucket during model training. If you want to use an existing bucket, you can skip this step.

***


In [ ]:
import boto3
import botocore

mySession = boto3.session.Session()
AwsRegion = mySession.region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")


training_bucket = f"custom-labels-jumpstart-{AwsRegion}-{account_id}"

In [ ]:
assets_bucket = 'jumpstart-cache-prod-us-east-1'
! aws s3 cp s3://$assets_bucket/ai_services_assets/custom_labels/cl_jumpstart_ic_notebook_utils.py utils.py

In [ ]:
from utils import create_bucket_if_not_exists
create_bucket_if_not_exists(training_bucket)    

#### 1.2.2. Set bucket permissions
***
Next, we set the S3 bucket permissions that allow Amazon Rekognition Custom Labels to access the S3 bucket and download the training images. For production applications, we recommend restricting thes permissions to only those needed for the application. For more information, see [Set up Amazon Rekognition Custom Labels permissions](https://docs.aws.amazon.com/rekognition/latest/customlabels-dg/su-console-policy.html). 

***

In [ ]:
%%time

def set_bucket_permissions(training_bucket):
    s3_client = boto3.client("s3")
    try:
        response_public = s3_client.put_public_access_block(Bucket=training_bucket, 
                        PublicAccessBlockConfiguration={"BlockPublicAcls": True, "IgnorePublicAcls": True,
                        "BlockPublicPolicy": True,"RestrictPublicBuckets": True})
    except Exception as e:
        raise Exception(f"Error setting bucket permissions. {str(e)}.")

set_bucket_permissions(training_bucket)

### 1.3. Install dependencies

Here we install dependencies needed to run the notebook.

In [ ]:
%%time
## Upgrade boto3 and botocore
!pip install botocore --upgrade
!pip install boto3 --upgrade
!pip install ipywidgets

In [ ]:
%%time
import json
from datetime import datetime
from os import listdir, makedirs
from os.path import isfile, join
import shutil

import boto3
from IPython.display import HTML, display, Image as IImage
from PIL import Image, ImageDraw, ImageFont
import time
import os

from IPython.display import JSON

s3_client = boto3.client('s3')

## 2. Set up the example datasets

***
We provide the following datasets that demonstrate the types of model that Custom Labels creates:

* ***Rooms:*** Creates a model that classifies images of household locations such as living room, bedroom, or backyard. Use this dataset to create an image classification model.

* ***Flowers:***  Creates a model that classifies images of flowers into multiple categories, such as the flower type and growth stage. Use this dataset to create a multi-label classification model.

* ***PCB:*** Creates a model that detects the location of circuit board parts on an image. Use this dataset to create an object detection model.

* ***Brand/Logo detection:*** Creates an object detection model that finds the locations of logos or brands in an image.


We provide the datasets in a format that can easily be used with the Custom Label APIs. The Input dataset folder has the following structure: 


* train
    * train_img1.jpg
    * train_img2.jpg
    * training_dataset.manifest

* test
    * test_img1.jpg
    * test_img2.jpg
    * testing_dataset.manifest

* validation
    * val_img1.jpg
    * val_img2.jpg



**train:** A set of labeled or annotated images Custom Labels uses these images to train the model. 
<br>
**test:** A set of labeled or annotated images that Custom Labels uses these images to evaluate the model, select the optimal hyper-parameters, and select the best training algorithm.
<br>
**validation:** A set of images that we use to try inference on the trained model.


Here are the number of train and test images for each of the dataset:

| Dataset     |  Train  |  Test |
| ----------- | -----   | ----- |
| Rooms       | 61      | 53    |
| Flowers     | 15      | 12    |
| PCB         | 16      | 12    |
| Brands/Logo | 33      | 5     |

training_dataset.manifest and testing_dataset.manifest are the manifest files that Custom Labels uses to train and test the model. A manifest file contains image locations and labels assigned to images.

Custom Labels requires Amazon Sagemaker Ground Truth format datasets.  If you want to use a dataset that's in a different format, such as COCO or a CSV file, see [Transforming Datasets from COCO format](https://docs.aws.amazon.com/rekognition/latest/customlabels-dg/md-transform-coco.html) or [Creating a manifest file from a CSV file](https://docs.aws.amazon.com/rekognition/latest/customlabels-dg/ex-csv-manifest.html). To learn more about manifest files, see [Creating a manifest file](https://docs.aws.amazon.com/rekognition/latest/customlabels-dg/md-create-manifest-file.html). 

***

### 2.1. Select and download a dataset

Here we select and download the dataset (Rooms, Flowers, or PCB) that we want to use for model training.

In [ ]:
from ipywidgets import Dropdown
import IPython

dataset_type_dropdown = Dropdown(
    options=["pcb (od)", "rooms (single label ic)", "flowers (multi label ic)", "brand_logo (od)"],
    value="pcb (od)",
    description="Select the dataset:",
    style={"description_width": "initial"},
    layout={"width": "max-content"},
)
display(IPython.display.Markdown("### Select a Dataset"))
display(dataset_type_dropdown)


In [ ]:
dataset_type = dataset_type_dropdown.value.split(' ')[0]

In [ ]:
# Download the dataset from the public bucket where dataset is previously downloaded and stored
downloaded_data_bucket = 'jumpstart-cache-prod-us-east-1'

if dataset_type == "pcb":
    downloaded_data_prefix = "ai_services_assets/custom_labels/datasets/pcb"
elif dataset_type == "rooms":
    downloaded_data_prefix = "ai_services_assets/custom_labels/datasets/rooms"
elif dataset_type == "flowers":
    downloaded_data_prefix = "ai_services_assets/custom_labels/datasets/multi_class_flowers"
elif dataset_type == "brand_logo":
    downloaded_data_prefix = "ai_services_assets/custom_labels/datasets/logos_object_detection"
    
data_folder = f"{dataset_type}_dataset"
!aws s3 cp --recursive s3://$downloaded_data_bucket/$downloaded_data_prefix $data_folder

### 2.2. Substitute the manifest file and upload the dataset
***
We provide template training and test manifest files which contain the labels/annotations for the images. The manifest files are already in the file format that Custom Labels needs to train a model. To use the manifest files, we do the following:
1. Substitute the placeholder S3 path in the training and test manifest files with the S3 bucket that we previously created.
2. Upload the manifest files to the previously created S3 bucket.
***

In [ ]:
%%time

placeholder_path = "<path to dataset>\/"
training_dataset_path = f'{training_bucket}\/{data_folder}\/train\/'
testing_dataset_path = f'{training_bucket}\/{data_folder}\/test\/'


! cp ./$data_folder/train/training_dataset.manifest ./$data_folder/train/training_dataset_cl.manifest
! cp ./$data_folder/test/testing_dataset.manifest ./$data_folder/test/testing_dataset_cl.manifest

# Substitute the placeholder path with s3 folder path where the training and test dataset is located.
! sed -i "s/$placeholder_path/$training_dataset_path/g" ./$data_folder/train/training_dataset_cl.manifest
! sed -i "s/$placeholder_path/$testing_dataset_path/g" ./$data_folder/test/testing_dataset_cl.manifest

In [ ]:
%%time

#Upload the data in the bucket for training.

s3train = f"s3://{training_bucket}/{data_folder}/train/"
s3test = f"s3://{training_bucket}/{data_folder}/test/"

! aws s3 cp ./$data_folder/train $s3train --recursive
! aws s3 cp ./$data_folder/test $s3test --recursive

### 2.3. Importing training and test datasets

***
Next, we create a project that you use to manage your datasets and the models that you train. We then create the datasets that Custom Labels uses to train and test the model.

We create an Amazon Rekognition Custom Labels Project by using the [create_project operation](https://docs.aws.amazon.com/rekognition/latest/APIReference/API_CreateProject.html). For more information, see [Creating a project](https://docs.aws.amazon.com/rekognition/latest/customlabels-dg/mp-create-project.html).
***

In [ ]:
%%time
rekognition = boto3.client('rekognition')
cl_project = rekognition.create_project(ProjectName=f"jumpstart-example-cl-nb-{dataset_type}-{str(int(time.time()))}")
JSON(cl_project)

***
Next, we import the training dataset into the Custom Labels project by using the [create_dataset](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/rekognition.html#Rekognition.Client.create_dataset) operation. We use the [describe_dataset](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/rekognition.html#Rekognition.Client.describe_datasetl) operation to track the status of the dataset creation.
The dataset is ready if the value of **Status** in **DatasetDescription** is CREATE_COMPLETE.

 For more information, see [Creating a dataset with a manifest file (SDK)](https://docs.aws.amazon.com/rekognition/latest/customlabels-dg/md-create-dataset-ground-truth-sdk.html).
***

In [ ]:
%%time
## We will use the dataset.manifest file which was created earlier, to create a training dataset.

train_manifest_key = f'{data_folder}/train/training_dataset_cl.manifest'
cl_dataset_train = rekognition.create_dataset(DatasetSource={"GroundTruthManifest": {
            "S3Object": {"Bucket": training_bucket, "Name": train_manifest_key}}},
            DatasetType="TRAIN", ProjectArn=cl_project["ProjectArn"])
JSON(cl_dataset_train)

In [ ]:
%%time
from utils import wait_dataset_creation_cl
dataset_status = wait_dataset_creation_cl(rekognition, cl_dataset_train)
JSON(dataset_status)

***
We import the test dataset in the same way that we create the training dataset.
***

In [ ]:
%%time
## We will a  test dataset.
test_manifest_key = f'{data_folder}/test/testing_dataset_cl.manifest'
cl_dataset_test = rekognition.create_dataset(DatasetSource={
        "GroundTruthManifest": {"S3Object": {"Bucket": training_bucket, "Name": test_manifest_key}}},
        DatasetType="TEST",ProjectArn=cl_project["ProjectArn"])
JSON(cl_dataset_test)

In [ ]:
dataset_status = wait_dataset_creation_cl(rekognition, cl_dataset_test)
JSON(dataset_status)

### 3. Train the model
***

**Important** You are charged for the amount of time it takes to successfully train a model.

***

#### 3.1. Start Training
***
Here we use the [create_project_version](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/rekognition.html#Rekognition.Client.create_project_version) operation to start training the model. For more information, see [Training an Amazon Rekognition Custom Labels model](https://docs.aws.amazon.com/rekognition/latest/customlabels-dg/training-model.html).
***


In [ ]:
%%time
## Start Training the model
model_version_name = f"jumpstart-example-cl-nb-model_v{str(int(time.time()))}"
project_folder = f"{dataset_type}_sample_project"
cl_train_model = rekognition.create_project_version(
    ProjectArn=cl_project["ProjectArn"], VersionName=model_version_name,
    OutputConfig={"S3Bucket": training_bucket, "S3KeyPrefix": f"{project_folder}/model_train"})
JSON(cl_train_model)

#### 3.1.1. Wait for training to finish

***
Training takes a while to finish. Here we wait until the training finishes, which might take 2 - 4  hours. Behind the scenes, Custom Label trains multiple models and selects the best performing model for you. Training has finished successfully if the value of Status in the JSON response from [describe_project_versions](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/rekognition.html#Rekognition.Client.describe_project_versions) is TRAINING_COMPLETED.
***

In [ ]:
%%time
## Wait for the training to finish. This might take 2 to 4 hours
training_completed_waiter = rekognition.get_waiter("project_version_training_completed")
training_completed_waiter.wait(ProjectArn=cl_project["ProjectArn"], VersionNames=[model_version_name])
model_traing_status = rekognition.describe_project_versions(ProjectArn=cl_project["ProjectArn"], VersionNames=[model_version_name])
JSON(model_traing_status)

#### 3.2. Retrieve the trained model metrics
***
Here we use the [describe_project_versions](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/rekognition.html#Rekognition.Client.describe_project_versions) operation to get the metrics for the trained model. The model results and metrics can also be viewed from the Amazon Rekognition Custom Labels Console. To learn more about the available metrics, see [Metrics for evaluating your model](https://docs.aws.amazon.com/rekognition/latest/customlabels-dg/im-metrics-use.html). To programatically access more granular results, see [Accessing Amazon Rekognition Custom Labels training results](https://docs.aws.amazon.com/rekognition/latest/customlabels-dg/im-metrics-api.html). 
***

In [ ]:
%%time
model_metrics = rekognition.describe_project_versions(ProjectArn=cl_project["ProjectArn"], VersionNames=[model_version_name])
# model_metrics

evaluation_result = model_metrics["ProjectVersionDescriptions"][0]["EvaluationResult"]
print("F1 Score " + str(evaluation_result["F1Score"]))

s3 = boto3.resource("s3")
evaluation_result_s3obj = evaluation_result["Summary"]["S3Object"]
content_object = s3.Object(evaluation_result_s3obj["Bucket"], evaluation_result_s3obj["Name"])
file_content = content_object.get()["Body"].read().decode("utf-8")
json_content = json.loads(file_content)

JSON(json_content)

### 4. Start the model and run inference 
***
To get predictions for an image (run inference), we first start the model. Once started, the model provides an endpoint that we use to pass an image to the model and get inference results back.  

**Important:** You are charged for the amount of time that your model runs.
***

#### 4.1. Start the trained model
***
Here we use the [start_project_version](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/rekognition.html#Rekognition.Client.start_project_version) API to start running the trained model. For more information, see [Starting an Amazon Rekognition Custom Labels model](https://docs.aws.amazon.com/rekognition/latest/customlabels-dg/rm-start.html).


***

In [ ]:
%%time
start_model = rekognition.start_project_version(
    ProjectVersionArn=model_metrics["ProjectVersionDescriptions"][0]["ProjectVersionArn"],
    MinInferenceUnits=1)
JSON(start_model)

In [ ]:
%%time
## Wait for the model to start

project_version_running_waiter = rekognition.get_waiter('project_version_running')
project_version_running_waiter.wait(ProjectArn=cl_project['ProjectArn'], VersionNames=[model_version_name])

model_start_status = rekognition.describe_project_versions(ProjectArn=cl_project['ProjectArn'], VersionNames=[model_version_name])

JSON(model_start_status)

#### 4.2. Run inference on the validation images
***
Here we use the [detect_custom_labels](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/rekognition.html#Rekognition.Client.detect_custom_labels) operation to run inference on the validation images. The code displays the images and overlays labels found by the model. If the model detects objects, the code displays bounding boxes around detected objects. The validation images were not used in the training or test datasets. For more information, see [Analyzing an image with a trained model](https://docs.aws.amazon.com/rekognition/latest/customlabels-dg/detecting-custom-labels.html). 
***

In [ ]:
%%time

import boto3
import io
import logging
from PIL import Image, ImageDraw, ImageFont
from IPython.display import HTML, display
from botocore.exceptions import ClientError

logger = logging.getLogger(__name__)


def get_image_bytes(photo):
    """Return the photo in bytes form."""
    image=Image.open(photo) 

    # get images bytes for call to detect_anomalies
    image_bytes = io.BytesIO()
    image.save(image_bytes, format=image.format)
    image_bytes = image_bytes.getvalue()
    return image_bytes


def show_image_with_response_cl(photo, response):
    """Displays the analyzed image and overlays analysis results

    :param photo: The analyzed photo
    :param response: the response from DetectCustomLabels
    """
    try:
        font_size, line_width = 20, 5

        image = Image.open(photo)
        img_width, img_height = image.size
        draw = ImageDraw.Draw(image)

        # calculate and display bounding boxes for each detected custom label
        image_level_label_height = 0

        for custom_label in response['CustomLabels']:
            confidence = int(round(custom_label['Confidence'], 0))
            label_text = f"{custom_label['Name']}:{confidence}%"
            fnt = ImageFont.truetype("/usr/share/fonts/dejavu/DejaVuSans.ttf", font_size)
            text_width, text_height = draw.textsize(label_text, fnt)

            logger.info(f"Label: {custom_label['Name']}")
            logger.info(f"Confidence:  {confidence}%")

            # Draw bounding boxes, if present
            if 'Geometry' in custom_label:
                box = custom_label['Geometry']['BoundingBox']
                left = img_width * box['Left']
                top = img_height * box['Top']
                width = img_width * box['Width']
                height = img_height * box['Height']

                logger.info("Bounding box: Left {0:.0f}".format(left) + ", Top {0:.0f}".format(top) +
                            ", Label Width {0:.0f}".format(width) + ", Label Height {0:.0f}".format(height))

                points = ((left, top), (left + width, top), (left + width, top + height),
                          (left, top + height), (left, top))
                # Draw bounding box and label text
                draw.line(points, fill="limegreen", width=line_width)
                draw.rectangle([(left + line_width, top + line_width),
                                (left + text_width + line_width, top + line_width + text_height)], fill="black")
                draw.text((left + line_width, top + line_width), label_text, fill="red", font=fnt)

                # draw image-level label text.
            else:
                draw.rectangle(
                    [(10, image_level_label_height), (text_width + 10, image_level_label_height + text_height)],
                    fill="black")
                draw.text((10, image_level_label_height), label_text, fill="red", font=fnt)

                image_level_label_height += text_height
        display(image)

    except Exception as err:
        logger.error(f'Error displaying images with response. Error: {str(err)}.')
        raise


def analyze_rekcl(photos, model_arn, min_confidence):
    """Invoke model and display the analysis results for each of the input photo.
    
    :param photos: A list of photo names.
    :param model_arn: model_arn for the custom label project
    :min_confidence: lower bound on the prediction confidence for predictions used in the analsis.
    """
    rek_client=boto3.client('rekognition')
    try:
        for photo in photos:
            image_bytes = get_image_bytes(photo)
            response = rek_client.detect_custom_labels(Image={'Bytes': image_bytes}, MinConfidence=min_confidence, ProjectVersionArn=model_arn)
            show_image_with_response_cl(photo, response)
            
    except ClientError as err:
        raise Exception("A service client error occurred: " + format(err.response["Error"]["Message"]))
    except ValueError as err:
        raise Exception("A value error occurred: " + format(err))

In [ ]:
%%time
## Test the model on the validation dataset
import glob
holdout_photos = glob.glob(f"{data_folder}/validation/*")
model_arn=model_metrics['ProjectVersionDescriptions'][0]['ProjectVersionArn']
min_confidence=60
analyze_rekcl(holdout_photos, model_arn, min_confidence)

### 4.3. Stop the trained model
***
 You are charged for the amount of time that your model runs. If you are not using the model, you should stop it running. Here we use the [stop_project_version](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/rekognition.html#Rekognition.Client.stop_project_version) operation to stop the model running. For more information, see [Stopping an Amazon Rekognition Custom Labels model](https://docs.aws.amazon.com/rekognition/latest/customlabels-dg/rm-stop.html). A model can also be stopped by using the Amazon Rekognition Custom Labels console. For more information, see see [4.2. Stopping Model through console](#4.2.-Stopping-Model-through-console).

***

In [ ]:
%%time
stop_model = rekognition.stop_project_version(ProjectVersionArn=model_metrics['ProjectVersionDescriptions'][0]['ProjectVersionArn'])
JSON (stop_model)

In [ ]:
%%time
from utils import wait_cl_model_stop
model_stop_status = wait_cl_model_stop(rekognition, cl_project, model_version_name)
JSON(model_stop_status)

You have successfully used the Amazon Rekognition Custom Label API to run end to end Custom Labels training and inference workflow. If you want to create a different type of model, start again from step 2 and select a different dataset. For more information, see [Additional resources](#5.3.-Additional-resources).

## 5. Conclusion

***
Custom Label provides easy-to-use ML workflow for creating different types of computer vision models, such as image classification and object detection. To train a model, you only need to provide labeled images that are suitable for your business needs. Custom Labels simplifies model training by taking care of parameter choices such as such as machine type, algorithm type, or algorithm specific hyperparameters (including the number of layers in the network, learning rate and batch size). Custom Labels simplifies hosting of a trained model model and provides a simple operation for performing inference with a trained model.

In this notebook, we showed you how to use the [Custom Label API](https://docs.aws.amazon.com/rekognition/latest/customlabels-dg/custom-labels-api-reference.html) to create different types of computer vision model. The notebook showed how to process data to create the training and test datasets, train a model, host a model, run inference, and stop a model.  To do this we provided example datasets for each of the model types: single label image classification (Rooms dataset), multi-label image classification (Flowers dataset) and object detection (PCB dataset). 
 
Custom Labels also provides an easy to use console experience for the training process, model management, and visualization of dataset images. 


***

## 6. Learn more about Rekognition Custom Labels
***
You can login to Amazon Rekognition Custom Labels console to visualize your training dataset, train your model, see predictions that the trained model makes the test dataset, and see evaluation metrics for the trained model. You can also start or stop your model from the Custom Labels console. 

***

### 6.1. Viewing dataset through console

Custom Labels provide a user-friendly [AWS console](https://console.aws.amazon.com/rekognition/custom-labels#/) that you can use to visualize the images and labels in training and test datasets. For more information, see [notebook](https://github.com/aws-samples/amazon-rekognition-code-samples/blob/main/custom-labels/custom-object-detection-using-rekognition-custom-labels-api.ipynb).
![custom_label_dataset_console](https://jumpstart-cache-prod-us-east-1.s3.us-east-1.amazonaws.com/ai_services_assets/custom_labels/screenshots/Dataset_6.png)

### 6.2. Stopping a model with the console
***
Please verify that the model stopped using the Rekogntion Custom labels to avoid incurring any cost. To stop a model by using the Amazon Rekogntion Custom Labels Console, see [Stop Custom Label model](https://docs.aws.amazon.com/rekognition/latest/customlabels-dg/gs-step-stop-model.html).
***

### 6.3. Additional resources
***

To learn more about the Custom Labels, see [Amazon Rekognition Custom Labels](https://docs.aws.amazon.com/rekognition/latest/customlabels-dg/what-is.html).

Custom Labels provides other example datasets that you can use in the console. For more information, see [Getting started with Amazon Rekognition Custom Labels](https://docs.aws.amazon.com/rekognition/latest/customlabels-dg/getting-started.html).

For a walkthrough of image classification with the Amazon Rekognition Custom Labels console, see [Tutorial: Classifying images](https://docs.aws.amazon.com/rekognition/latest/customlabels-dg/tutorial-classification.html).

To learn more about the Custom Labels API, please see [Amazon Rekognition Custom Labels API reference](https://docs.aws.amazon.com/rekognition/latest/customlabels-dg/custom-labels-api-reference.html).

For more examples, see the [Sample Rekognition Custom label Notebooks](https://github.com/aws-samples/amazon-rekognition-code-samples/tree/main/custom-labels). 
***